In [7]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from string import punctuation
from collections import defaultdict

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
BASE_DIR = '../data/'
GLOVE_DIR = BASE_DIR + 'interim/'
# TRAIN_DATA_FILE = BASE_DIR + 'processed/train.csv'
# TEST_DATA_FILE = BASE_DIR + 'processed/test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.01

In [3]:
num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

In [4]:
act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [5]:
print('Indexing word vectors.')
embeddings_index = {}
f = codecs.open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [6]:
df_data = pd.read_csv('../data/processed/featured_data.csv')

In [8]:
df_data.columns

Index(['id', 'is_duplicate', 'common_word_ratio', 'fuzz_ratio',
       'Partial_Ratio', 'Token_Sort_Ratio', 'Token_Set_Ratio', 'cleanQ1',
       'cleanQ2'],
      dtype='object')

In [9]:
X = df_data.drop(['id', 'is_duplicate'], axis=1)
y = df_data['is_duplicate']

X_train,X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state = 42)



In [33]:
token_text = pd.concat([X_train['cleanQ1'], X_train['cleanQ2'], X_test['cleanQ1'], X_test['cleanQ2']])

    

In [35]:
# token_text = X_train['cleanQ1'].values + X_train['cleanQ2'].values + X_test['cleanQ1'].values +  X_test['cleanQ2'].values
print(len(token_text))
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(token_text.values)

sequences_1 = tokenizer.texts_to_sequences(X_train['cleanQ1'])
sequences_2 = tokenizer.texts_to_sequences(X_train['cleanQ2'])
test_sequences_1 = tokenizer.texts_to_sequences(X_test['cleanQ1'])
test_sequences_2 = tokenizer.texts_to_sequences(X_test['cleanQ2'])

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(y_train)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

323164
Found 38558 unique tokens
Shape of data tensor: (113107, 30)
Shape of label tensor: (113107,)


In [36]:
test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_labels = np.array(y_test)

In [37]:
########################################
## prepare embeddings
########################################
print('Preparing embedding matrix')

nb_words = max(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 178203


In [38]:
########################################
## sample train/validation data
########################################
#np.random.seed(1234)
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train]))
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [39]:
########################################
## define the model structure
########################################
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [40]:
########################################
## add class weight
########################################
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

In [41]:
########################################
## train the model
########################################
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        epochs=20, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

lstm_257_105_0.22_0.22
Train on 223950 samples, validate on 2264 samples
Epoch 1/20
223950/223950 [==============================] - 492s 2ms/step - loss: 0.4933 - acc: 0.6683 - val_loss: 0.4310 - val_acc: 0.6299
Epoch 2/20
223950/223950 [==============================] - 489s 2ms/step - loss: 0.3776 - acc: 0.6932 - val_loss: 0.3989 - val_acc: 0.6396
Epoch 3/20
223950/223950 [==============================] - 489s 2ms/step - loss: 0.3549 - acc: 0.7103 - val_loss: 0.3740 - val_acc: 0.6528
Epoch 4/20
223950/223950 [==============================] - 488s 2ms/step - loss: 0.3396 - acc: 0.7251 - val_loss: 0.3489 - val_acc: 0.7049
Epoch 5/20
223950/223950 [==============================] - 488s 2ms/step - loss: 0.3269 - acc: 0.7364 - val_loss: 0.3459 - val_acc: 0.7394
Epoch 6/20
223950/223950 [==============================] - 489s 2ms/step - loss: 0.3148 - acc: 0.7483 - val_loss: 0.3316 - val_acc: 0.7628
Epoch 7/20
223950/223950 [==============================] - 490s 2ms/step - loss: 0.304

In [42]:
preds = model.predict([test_data_1, test_data_2], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1], batch_size=8192, verbose=1)
preds /= 2

48475/48475 [==============================] - 36s 743us/step


In [44]:
print(preds.ravel())

[0.24714516 0.36276057 0.5838347  ... 0.3712837  0.219223   0.03184133]
